<table style="width: 100%;" id="nb-header>">
        <tr style="background-color: transparent;"><td>
            <img src="https://ds-connectors.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
        </td><td>
            <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2020<br>
                Dr. Eric Van Dusen<br>
            Notebook by Chris Pyles</p></td></tr>
    </table>

# Lab 9: Game Theory and the Prisoner's Dilemma

Project 2 will deal with game theory, a subdomain of economics and statistics that considers the best strategies and equilibria for playing games. In particular, you will consider one of the most famous thought experiments in game theory: the prisoner's dilemma. In this lab, we will introduce this game and some strategies for playing it, in preparation for the project.

In [ ]:
from datascience import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("seaborn-muted")
import functools
import nbforms
feedback = nbforms.Form("feedback_form_config.json")
feedback.take_attendance()

most_common = lambda a: a[np.argmax([np.count_nonzero(np.equal(i, a)) for i in a])]

def create_player_class(player_name, play_method):
    @functools.total_ordering
    class Player:
        def __init__(self, p=0.5):
            self.history = make_array()
            self.prob = p
            self.name = player_name
        
        def play(self, opponent, is_p1):
            """Returns True if player defects, False otherwise"""
            defect = play_method(self, opponent, is_p1)
            self.history = np.append(self.history, defect)
            return defect
        
        def reset_history(self):
            self.history = make_array()
        
        def __repr__(self):
            if player_name == "Random":
                return player_name + "({})".format(self.prob)
            return player_name
        
        def __hash__(self):
            return hash(self.name)
        
        def __eq__(self, other):
            return self.name == other.name
        
        def __lt__(self, other):
            return self.name < other.name
    
    return Player

The [prisoner's dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) is a classic game first discussed by Merrill Flood and Melvin Dresher in 1950. In this game, there are two prisoners who have been captured and are being interrogated. The prinsoners cannot contact each other in any way. They have two options: they can **defect** (betray the other prisoner to the police) or they can **cooperate** (maintain their silence). If both defect, both receive 4 years in prinson. If one defects and the other does not, the defector goes free and the cooperator receives 5 years in prison. If both cooperate (meaning neither talks to the police), then they each receive 2 years in prison.

<table>
    
<tr style="background-color: white;"><td></td><td></td><td colspan="2">Prisoner A</td></tr>
<tr><td></td><td></td><td>Cooperate</td><td>Defect</td></tr>
<tr style="background-color: white;"><td rowspan="2">Prisoner B</td><td>Cooperate</td><td style="background-color: #F5F5F5;">A: 2, B: 2</td><td>A: 0, B: 5</td></tr>
<tr><td>Defect</td><td>A: 5, B: 0</td><td style="background-color: #F5F5F5;">A: 4, B: 4</td></tr>
    
</table>

The purpose of this game is to consider how a completely rational person would be best advised to proceed, and how different strategies for playing this game can be more or less effective. In this project, we will be studying the effectiveness of different strategies for playing this game using tournaments, in which the game is played in multiple rounds to see which player has the best payoff.

In this lab, we will be creating some custom Python classes to represent different playing strategies. We'll be using the `create_player_class` function, defined above, to create these classes for us. Here are a few important things to know about the classes:

Consider a player class instance `player = Player()`. 
* `player.play()` is a method that represents a single move. It returns `True` if the player **defects** and `False` if they cooperate.
* `player.history` is an array of the previous moves of the player instance. `player.history.item(-1)`, for example, is the most recent move (the last element of the array).

Don't worry about the other methods that are defined for players; these are there for the code below to work but you don't need to concern yourself with them. Here's how the `create_player_class` function works:

1. Define a method, `f`, that takes three arguments: `self`, `opponent`, and `is_p1` (more about these later), and returns `True` if the player defects and `False` otherwise.
2. Call `create_player_class` with two arguments: a string for the class name, e.g. `"Player"`, and the play method.

Let's illustrate this by creating `Defector`, a player that always defects. The `defector_play` function below always returns `True`, since the player always defects. We then create `Defector` using `create_player_class`.

In [ ]:
def defector_play(self, opponent, is_p1):
    return True

Defector = create_player_class("Defector", defector_play)

We can create a `Defector` by calling `Defector` as if it were a function:

In [ ]:
Defector()

**Question 1:** Create a `Cooperator` class that always cooperates.

In [ ]:
def cooperator_play(self, opponent, is_p1):
    return ...

Cooperator = create_player_class(..., ...)

Recall the payoff structure of our game:

<table>
    
<tr style="background-color: white;"><td></td><td></td><td colspan="2">Prisoner A</td></tr>
<tr><td></td><td></td><td>Cooperate</td><td>Defect</td></tr>
<tr style="background-color: white;"><td rowspan="2">Prisoner B</td><td>Cooperate</td><td style="background-color: #F5F5F5;">A: 2, B: 2</td><td>A: 0, B: 5</td></tr>
<tr><td>Defect</td><td>A: 5, B: 0</td><td style="background-color: #F5F5F5;">A: 4, B: 4</td></tr>
    
</table>

Let's define a function to calculate the payoff of our game for a single move.

**Question 2:** Fill in the `payoff` function below that calls `.play()` for each player (with the appropriate arguments!) and returns the payoff for `p1` and `p2` as a `tuple`.

_Python Note:_ Python classes automatically fill in the `self` argument, so when calling `player.play()`, you only need two arguments: `opponent` and `is_p1`.

In [ ]:
def payoff(p1, p2):
    p1_defect = p1.play(...)
    p2_defect = p2.play(...)
    
    if ...:
        return 4, 4
    elif ...:
        return 0, 5
    elif ...:
        return 5, 0
    else:
        return 2, 2
    
payoff(Defector(), Cooperator())

Now that we can look at payoffs, let's consider another strategy: randomness. The `Random` player defined below will randomly defect or cooperate. Note that the `Random()` constructor takes an optional argument indicating the probability of **defecting** on any given turn which defaults to 0.5. For example, `Random(.25)` defects 25% of the time and `Random(1)` always defects (it's a `Defector`).

In [ ]:
def random_play(self, opponent, is_p1):
    return np.random.random() < self.prob

Random = create_player_class("Random", random_play)

Notice in the `random_play` that we access an instance variable of `self`, `self.prob`. The purpose of the first argument of these functions is to allow us to access variables within classes while playing, which will be of use later. Similarly, we can use the `opponent` argument to access variables of the opponent.

Let's test a few payoffs with our new `Random` player.

In [ ]:
payoff(Defector(), Random())

In [ ]:
payoff(Defector(), Random(.75))

Our analysis of the prisoner's dilemma hinges on analyzing strategies over multiple rounds. For this reason, we need a way to pit players against each other and find out who has the fewest years accrued. The function `run_match` below will run a match of two players with `n` turns. If `winner` is `True`, it returns the winner of the match; if `winner` is `False`, it returns a list for each player containing the sequence of years accrued. Also note the first two lines of the function that reset the player history for each player. This is important because we might end up reusing instances of players, and we don't want histories from past matches to affect the results of this match.

In [ ]:
def run_match(p1, p2, n=5, winner=True):
    p1.reset_history()
    p2.reset_history()
    
    p1_years = []
    p2_years = []
    
    for i in range(n):
        p1_score, p2_score = payoff(p1, p2)
        p1_years.append(p1_score)
        p2_years.append(p2_score)
        
    if winner:
        p1_mean = np.mean(p1_years)
        p2_mean = np.mean(p2_years)

        if p1_mean < p2_mean:
            return p1
        else:
            return p2
    else:
        return p1_years, p2_years

Let's take a look at what's returned for each value of `winner`. Note that it defaults to `True`.

In [ ]:
run_match(Defector(), Cooperator())

In [ ]:
run_match(Defector(), Cooperator(), winner=False)

Now that we know how to run matches, let's think of some other cool strategies. Remember the `player.history` variable discussed earlier? Let's have a concrete example on how to use that. The `Alternator` player defined below alternates between cooperating and defecting, beginning with cooperating.

Note the `if` statement. We need to check that we have a history to index into before we try to grab the last element; if we didn't have that statement, the code would fail on the first time  `Alternator` played, because we would try to get an element from an array with 0 elements.

In [ ]:
def alternator_play(self, opponent, is_p1):
    if len(self.history) > 0:
        return not self.history.item(-1)
    else:
        return False
    
Alternator = create_player_class("Alternator", alternator_play)

run_match(Alternator(), Defector())

**Question 3:** Use the player history to define `TitForTat`, a player that does the last thing it's opponent did and starts off by cooperating. For example, if we pitted `TitForTat` against a `Defector` in a 5-round match, the `Defector`'s moves would be `[D, D, D, D, D]`, and `TitForTat`'s would be `[C, D, D, D, D]`.

_Hint:_ Recall the `is_p1` argument. How is the result of `tit_for_tat_play` different if we are player 1 vs. player 2?

In [ ]:
def tit_for_tat_play(self, opponent, is_p1):
    if ...:
        if len(opponent.history) > 0 and ...:
            return ...
        else:
            return ...
    else:
        if len(opponent.history) > 1 and ...:
            return ...
        else:
            return ...
    
TitForTat = create_player_class(..., ...)

Now that we can pit two players against each other, how do we compare multiple players? We can create a tournament that runs matches between each pair of players.

**Question 4:** In a tournament with $n$ players, how many matches are run?

_Type your answer here, replacing this text._

To run through the tournament, we'll need to iterate through each player and pit them against every other player in a match. To ensure that we don't duplicate matches, we'll loop from 0 to the number of players in the outer loop and loop from the next player to the last player in the inner loop. The function `most_common` returns the value in an array of values that has the largest number of occurrences; it is here used to determine which player won the most times.

In [ ]:
players = make_array(Defector(), Cooperator())
winners = make_array()
for i in range(len(players)):
    for j in range(i+1, len(players)):
        winner = run_match(players.item(i), players.item(j))
        winners = np.append(winners, winner)
        
biggest_winner = most_common(winners)
biggest_winner

**Question 5:** Consider the four strategies we've defined so far: defector, cooperator, alternator, and tit-for-tat. Which strategy do you think would win in a tournament between these four? Why?

_Type your answer here, replacing this text._

**Question 6:** Fill in the code below to run your own tournament with the four non-random players we've defiend so far.

In [ ]:
players = make_array(...)
winners = make_array()
for i in range(...):
    for j in range(..., ...):
        winner = ...
        winners = np.append(...)
        
biggest_winner = most_common(winners)
biggest_winner

Instead of just determining the winner, let's collect some data. The modified tournament below collects the players in each match and their mean scores (note the user of `winner=False` in our `run_match` call). The table `results` contains a row for each match in  the tournament.

In [ ]:
players = make_array(Defector(), Cooperator(), TitForTat(), Alternator())
results = Table().with_columns(
    "p1", make_array(),
    "p1_mean", make_array(),
    "p2", make_array(),
    "p2_mean", make_array()
)


for i in range(len(players)):
    for j in range(i+1, len(players)):
        p1_years, p2_years = run_match(players.item(i), players.item(j), winner=False)
        results = results.with_row([
            players.item(i),
            np.mean(p1_years),
            players.item(j),
            np.mean(p2_years)
        ])
        
results

How do we determine the winner of each match?

**Question 7:** Fill in the function `determine_winner` below that takes in two players and their mean scores and returns the winning player. Break ties arbitrarily.

In [ ]:
def determine_winner(p1, p1_mean, p2, p2_mean):
    ...

In the cell below, we apply the `determine_winner` function to our `results` table. Note how `Table.apply` allows you to use multiple columns corresponding to each argument. The call below will apply `determine_winner` to the values in `p1`, `p1_mean`, `p2`, and `p2_mean` (in that order) to each row of `results`.

In [ ]:
winners = results.apply(determine_winner, "p1", "p1_mean", "p2", "p2_mean")
results = results.with_column("winner", winners)
results

In the last section of this lab, we'll have a refresher on hypothesis testing. Recall that hypothesis testing, a statistical technique based on bootstrapping, is a method that allows us to run a test over many iterations to look for differences between two distributions. In this lab, we'll be bootstrapping tournaments to test whether randomness is a good strategy for playing the game.

Let's start by stating our null and alternative hypotheses.

**Question 8:** Of our three main strategies (tit-for-tat, alternator, and random), which do you think will win? Why?

_Type your answer here, replacing this text._

**Question 9:** Based on your answer to Question 8, what are the null and alternative hypotheses for the hypothesis test?

**Null hypothesis:** _Type your answer here, replacing this text._

**Alternative hypothesis:** _Type your answer here, replacing this text._

Now that that's out of the way, let's write some code.

**Question 10:** Fill in the cell below to run a hypothesis test on tournaments using our three main strategies: tit-for-tat, alternator, and random. To start, let the random player have an equal chance of cooperating and defecting.

_Hint:_ What should `winner` be set to in `run_match`?

In [ ]:
winners = make_array()
n = 1000

for i in np.arange(n):
    players = make_array(TitForTat(), Alternator(), Random())
    results = Table().with_columns(
        "p1", make_array(),
        "p1_mean", make_array(),
        "p2", make_array(),
        "p2_mean", make_array()
    )
    for i in range(...):
        for j in range(..., ...):
            p1_years, p2_years = ...
            results = results.with_row([
                players.item(i),
                np.mean(...),
                players.item(j),
                np.mean(...)
            ])
    tournament_winners = results.apply(...)
    results = results.with_column(...)    
    winner = results.group("winner").sort("count", descending=True).column(0).item(0)
    winners = np.append(winners, winner)

biggest_winner = most_common(winners)
biggest_winner

**Question 11:** Who was the biggest winner in our hypothesis test? Which hypothesis do we adopt?

_Type your answer here, replacing this text._

---

### Feedback

Please fill out the feedback form below regarding this lecture.

In [ ]:
feedback.ask()